In [1]:
import pandas as pd
from collections import defaultdict
import os
from tqdm import tqdm

In [2]:
# ID = "hsa-let-7a-1"


def Fetch_miRNA_Target(ID):

    URL = f"https://mirbase.org/cgi-bin/mirna_entry.pl?acc={ID}"
    ##################3

    dfs = pd.read_html(URL)
    
    # print(dfs)

    #########################


    # ID = {'ID':''}
    Mat = defaultdict(dict)

    Mat_temp = ''

    for i in range(len(dfs)):
        # print(">", i)

        M3 = ''
        M5 = ''

        df = dfs[i]
        Data = df.values.tolist()
        for J in range(len(Data)):
            j = Data[J]

            if len(j)==2:
                if "Stem-loop sequence" in j[0] and "Stem-loop sequence" in j[1]:
                    Mat['ID'] = j[0].replace('Stem-loop sequence ', '')
                if "Mature sequence" in j[0] and "Mature sequence" in j[1]:
                    Mat_temp = j[0].replace('Mature sequence ', '')
                    # print(Mat_temp)
                    Acc = Data[J+1][1]
                    # print(Acc)
                    Mat[Mat_temp] = Acc

                    if Mat_temp.endswith('5p'):
                        Mat['5p'] = Acc
                    if Mat_temp.endswith('3p'):
                        Mat['3p'] = Acc
    ########################

    df = pd.DataFrame.from_dict(Mat, orient='index')

    URL_5 = f"https://mirdb.org/cgi-bin/search.cgi?searchType=miRNA&full=mirbase&searchBox={Mat['5p']}"
    URL_3 = f"https://mirdb.org/cgi-bin/search.cgi?searchType=miRNA&full=mirbase&searchBox={Mat['3p']}"
    
    df_5 = pd.read_html(URL_5)[0]
    df_3 = pd.read_html(URL_3)[0]
    
    df_3.columns = df_3.iloc[0]
    df_3 = df_3[1:]
    df_3

    df_5.columns = df_5.iloc[0]
    df_5 = df_5[1:]
    df_5
    
    # df_5 = df_5.columns = df.iloc[0]
    # df_3 = df_3.columns = df.iloc[0]
    
    return df, df_3, df_5, Mat

# _ = Fetch_miRNA_Target(ID)

In [3]:
# Mat

In [5]:
# df, df_3, df_5, Mat = Fetch_miRNA_Target(ID)

In [6]:

directory = "miTargets/"
if not os.path.exists(directory):
    os.makedirs(directory)


In [7]:
def main(ID):
    print(ID)
    try:
        df, df_3, df_5, Mat = Fetch_miRNA_Target(ID)
        if len(df):
            Info_F = f"{directory}{Mat['ID']}_info.csv"
            df.to_csv(Info_F, index=False)

            p3_F = f"{directory}{Mat['ID']}_{Mat['3p']}_targets.tsv"
            df_3.to_csv(p3_F, index=False, sep="\t")


            p5_F = f"{directory}{Mat['ID']}_{Mat['5p']}_targets.tsv"
            df_5.to_csv(p5_F, index=False, sep="\t")
        else:
            print(f"{ID} Error")
    except:
        print(f"Error {ID}")

In [8]:
IDs = list(set(open('Input_list.txt').read().splitlines()))
IDs

['hsa-let-7a-1', 'hsa-let-7a-2', 'hsa-let-7b', 'hsa-let-7a-3']

In [9]:
for i in tqdm(range(len(IDs))):
    ID = IDs[i]
    main(ID)
           

  0%|                                                     | 0/4 [00:00<?, ?it/s]

hsa-let-7a-1


 25%|███████████▎                                 | 1/4 [00:02<00:07,  2.66s/it]

hsa-let-7a-2


 50%|██████████████████████▌                      | 2/4 [00:04<00:04,  2.32s/it]

hsa-let-7b


 75%|█████████████████████████████████▊           | 3/4 [00:06<00:02,  2.28s/it]

hsa-let-7a-3


100%|█████████████████████████████████████████████| 4/4 [00:09<00:00,  2.31s/it]


In [10]:
!ls {directory}

hsa-let-7a-1_info.csv		       hsa-let-7a-3_info.csv
hsa-let-7a-1_MIMAT0000062_targets.tsv  hsa-let-7a-3_MIMAT0000062_targets.tsv
hsa-let-7a-1_MIMAT0004481_targets.tsv  hsa-let-7a-3_MIMAT0004481_targets.tsv
hsa-let-7a-2_info.csv		       hsa-let-7b_info.csv
hsa-let-7a-2_MIMAT0000062_targets.tsv  hsa-let-7b_MIMAT0000063_targets.tsv
hsa-let-7a-2_MIMAT0010195_targets.tsv  hsa-let-7b_MIMAT0004482_targets.tsv


# END